From https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html

In [6]:
# YOLO object detection
import cv2 as cv
import numpy as np
import time
import os

In [7]:
WHITE = (255, 255, 255)
img = None
img0 = None
outputs = None

# Load names of classes and get random colors
    # classes = open('coco.names').read().strip().split('\n')

classes = open('classes.txt').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

In [8]:
# Give the configuration and weight files for the model and load the network.
net = cv.dnn.readNetFromDarknet('yolov3_CUSTOM.cfg', 'yolov3_CUSTOM_last.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

Functions

In [9]:
def load_image(path):
    global img, img0, outputs, ln

    img0 = cv.imread(path)
    img = img0.copy()
    
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time() - t0

    # combine the 3 output groups into 1 (10647, 85)
    # large objects (507, 85)
    # medium objects (2028, 85)
    # small objects (8112, 85)
    outputs = np.vstack(outputs)

    post_process(img, outputs, 0.5)
    cv.imshow('window',  img)
    cv.displayOverlay('window', f'forward propagation time={t:.3}')
    cv.waitKey(0)

def post_process(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores) # which class
        confidence = scores[classID] # confidence
        
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            cv.rectangle(img, p0, p1, WHITE, 1)
            
    indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            print(classes[classIDs[i]], confidences[i])
        

def trackbar(x):
    global img
    conf = x/100
    img = img0.copy()
    post_process(img, outputs, conf)
    cv.displayOverlay('window', f'confidence level={conf}')
    cv.imshow('window', img)



In [10]:
assert False

AssertionError: 

Tests

In [ ]:
cv.namedWindow('window')
cv.createTrackbar('confidence', 'window', 50, 100, trackbar)

load_image('test_images/anemone_test_1.jpeg') # seastar
# load_image('test_anemone.jpeg') # anemone
# load_image('test_nudibranch.jpg') # nudibranch
# load_image('test_barnacle.jpg') # barnacle
# load_image('test_bivalve.jpg') # bivalve
# load_image('test_mussel.jpg') # bivalve
# load_image('test_mussel_6.jpeg') # bivalve
# load_image('test_crab_7.jpeg') # crab
# load_image('test_crab_5.png') # crab


cv.destroyAllWindows()

anemone 0.9926396608352661


In [ ]:
def load_image_test(path):
    global img, img0, outputs, ln

    img0 = cv.imread(path)
    img = img0.copy()
    
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time() - t0
    outputs = np.vstack(outputs)

    post_process_test(img, outputs, 0.5)
#     cv.imshow('window',  img)
#     cv.displayOverlay('window', f'forward propagation time={t:.3}')
    cv.waitKey(0)
    

def post_process_test(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores) # which class
        confidence = scores[classID] # confidence
        
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            # cv.rectangle(img, p0, p1, WHITE, 1)
            
    indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            print(classes[classIDs[i]], confidences[i])
            
            if classes[classIDs[i]].isin(img):
                print('yes')

In [ ]:
# img_1 = '/users/noah/Github_repos/Project_5/test_images/anemone_test_1.jpeg'
# img_2 = '/users/noah/Github_repos/Project_5/test_images/barnacle_test_1.jpg'

# test_pics = [img_1, img_2]

# for pic in test_pics:
#     print(pic[47:])
#     load_image_test(pic)
#     print('\n')

In [ ]:
def test_all(test_dir_path):
    os.listdir(test_dir_path)
    for pic in os.listdir(test_dir_path):
        if pic.endswith('.jpeg') or pic.endswith('.jpg') :
            print(pic)
    #         print(test_dir_path+pic)
            load_image_test(test_dir_path+pic)
            print('\n')

In [ ]:
test_dir_path = '/users/noah/Github_repos/Project_5/test_images/'

test_all(test_dir_path)